In [12]:
# no imports start from scratch
colors = ('red','blu','grn')
numbers = ('one','two','thr')
shapes = ('circle','square','squigg')
shades = ('full','line','blnk')

In [13]:
my_deck = list()
for col in colors:
    for num in numbers:
        for shp in shapes:
            for shd in shades:
                my_deck.append((col,num,shp,shd))
            

In [25]:
from random import shuffle
new_deck = shuffle(my_deck)

In [45]:
foo  =[x for x in range(3)]
bar = shuffle(foo)

In [58]:
def get_deck(n_deck=1):
    # constants
    colors = ('red','blu','grn')
    numbers = ('one','two','thr')
    shapes = ('circle','square','squigg')
    shades = ('full','line','blnk')
    my_deck = list()
    for col in colors:
        for num in numbers:
            for shp in shapes:
                for shd in shades:
                    my_deck.append((col,num,shp,shd))
    deck_out = [sample(my_deck,len(my_deck)) for i in range(n_deck)]
    return deck_out

In [77]:
def deal_deck(deck,cards_needed = 12):
    n_missing = cards_needed- len(deck)
    if (n_missing<0): n_missing=0
    cards_needed -= n_missing
    #print(cards_needed)
    return deck[:cards_needed]

In [79]:
d1 = get_deck()
hand = deal_deck(d1[0])
hand

12


[('blu', 'one', 'circle', 'blnk'),
 ('red', 'thr', 'square', 'full'),
 ('grn', 'one', 'squigg', 'line'),
 ('blu', 'one', 'squigg', 'line'),
 ('grn', 'thr', 'squigg', 'full'),
 ('blu', 'two', 'circle', 'line'),
 ('blu', 'thr', 'squigg', 'blnk'),
 ('blu', 'one', 'square', 'blnk'),
 ('red', 'one', 'square', 'blnk'),
 ('grn', 'one', 'square', 'full'),
 ('red', 'two', 'circle', 'full'),
 ('grn', 'one', 'circle', 'full')]

In [115]:
def subset(items): 
    # checks if 3 items (i.e colors of three cards) constitute a set
    assert len(items)==3
    if (items[0]==items[1]==items[2]) or (items[0]!=items[1]!=items[2]!=items[0]): return True
    else: return False
def SET(cards):
    # checks if 3 cards constitute a set
    assert len(cards)==3 # must be 3 cards
    assert len(cards[0])==4 # normal set cards
    for quality in range(len(cards[0])):
        items = [x[quality] for x in cards]
        is_good = subset(items)
        if not is_good: return False # if Fail for any quality
    return True

In [100]:
tmp1 = 'cat'
tmp2 = 'dog'
tmp3 = 'dog'
tmp3==tmp3==tmp3
tmp2!=tmp1!=tmp1

False

In [193]:
def search_hand_brute_dumb(hand):
    # this method just looks at every hand in the deck starting with 1,2,3
    # dumb implies it will repeat hands by iterating but this should actually be easy
    # to eliminate by keeping a list of hands exausted in the first position
    n_cards = len(hand)
    good_c1 = [i for i in range(n_cards-2)]
    good_c2 = [i for i in range(1,n_cards-1)]
    good_c3 = [i for i in range(2,n_cards-1)]
    cnt = 0
    for c1 in good_c1:
        if any(x==c1 for x in good_c2):
            good_c2.remove(c1)
        if any(x==c1 for x in good_c3):
            good_c3.remove(c1)
        for c2 in good_c2:
            good_c3_use = [x for x in good_c3 if x>c2]
            for c3 in good_c3_use:
                c_use = [c1,c2,c3]
                cards = [hand[i] for i in c_use]
                cnt +=1
                is_set = SET(cards)
                if is_set:
                    return True, cards, c_use, cnt
    return False, [],[],cnt